# Chains in LangChain（LangChain中的链）

## Outline

* LLMChain（大语言模型链）
* Sequential Chains（顺序链）
  * SimpleSequentialChain
  * SequentialChain
* Router Chain（路由链）

### 为什么我们需要Chains ？
链允许我们将多个组件组合在一起，以创建一个单一的、连贯的应用程序。链（Chains）通常将一个LLM（大语言模型）与提示结合在一起，使用这个构建块，您还可以将一堆这些构建块组合在一起，对您的文本或其他数据进行一系列操作。例如，我们可以创建一个链，该链接受用户输入，使用提示模板对其进行格式化，然后将格式化的响应传递给LLM。我们可以通过将多个链组合在一起，或者通过将链与其他组件组合在一起来构建更复杂的链。

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:

import sys
sys.path.append("D:\\software\\conda\\envs\\torch2\\Lib\\site-packages")

In [ ]:
!pip install pandas
from langchain.chains import LLMChain


  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s  0:00:02m 6.8 MB/s eta 0:00:01
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]━━━━ 3/4 [pandas]


这些链的一部分的强大之处在于你可以一次运行它们在许多输入上，因此，我们将加载一个pandas数据框架

In [1]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [2]:
df.head()

,Product,Review
0,cea,1212


## 1. LLMChain

LLMChain是一个简单但非常强大的链，也是后面我们将要介绍的许多链的基础。

In [4]:
# from langchain.chat_models import ChatOpenAI    #导入OpenAI模型
# from langchain.prompts import ChatPromptTemplate   #导入聊天提示模板
# from langchain.chains import LLMChain    #导入LLM链。
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import os

初始化语言模型

In [28]:
import os



In [33]:
# 初始化组件
# 移除或禁用可能会导致验证错误的不受支持代理（例如 socks://）
for proxy_var in ("ALL_PROXY", "all_proxy", "HTTP_PROXY", "http_proxy", "HTTPS_PROXY", "https_proxy"):
    os.environ.pop(proxy_var, None)

llm  = ChatOpenAI(
    temperature=0.7,
    model="glm-4-air-0111",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

llm2  = ChatOpenAI(
    temperature=0.7,
    model="glm-4.6",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

初始化prompt，这个prompt将接受一个名为product的变量。该prompt将要求LLM生成一个描述制造该产品的公司的最佳名称

In [6]:
prompt = ChatPromptTemplate.from_template(   
    "What is the best name to describe \
    a company that makes {product}?"
)

将llm和prompt组合成链---这个LLM链非常简单，他只是llm和prompt的结合，但是现在，这个链让我们可以以一种顺序的方式去通过prompt运行并且结合到LLM中

In [7]:
# chain = LLMChain(llm=llm, prompt=prompt)

# 进阶:使用LCEL(LangChain Expression Language)链式调用
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt
    | llm
    
)

因此，如果我们有一个名为"Queen Size Sheet Set"的产品，我们可以通过使用chain.run将其通过这个链运行

In [8]:
product = "Queen Size Sheet Set"
chain.invoke(product)

AIMessage(content='Okay, let\'s break down what makes a "best" name. It depends on what you want to emphasize:\n\n*   **Comfort & Luxury:** If your sheets are high-end, soft, and plush.\n*   **Quality & Durability:** If you want to emphasize longevity and a great value.\n*   **Style & Design:** If your sheets have unique patterns or colors.\n*   **Simplicity & Clarity:** If you want to be straightforward about what you sell.\n*   **Memorability & Catchiness:** If you want something easy to remember and say.\n\nHere are some name ideas, categorized by the angle they take:\n\n**Focusing on Comfort & Luxury:**\n\n1.  **Velvet Queen:** Evokes softness and luxury.\n2.  **Cocoon Queen:** Suggests comfort and being wrapped up.\n3.  **Luxe Queen:** Simple, clear, and points to luxury.\n4.  **Serene Sheets:** Focuses on the peaceful feeling of good sheets.\n5.  **Plush Queen:** Directly mentions softness.\n6.  **The Queen\'s Comfort:** A slightly regal, descriptive name.\n7.  **Cloud Queen:** I

您可以输入任何产品描述，然后查看链将输出什么结果

In [9]:
# 中文
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain = (llm | prompt)
product = "蓝牙耳机"
chain.invoke(product)

ChatPromptValue(messages=[HumanMessage(content="描述制造content='蓝牙耳机是一种无需线缆连接的无线耳机，它通过蓝牙技术与手机、电脑、平板等设备进行配对，实现音频的无线传输。蓝牙耳机具有便携、方便、舒适等优点，越来越受到消费者的喜爱。\\n\\n以下是一些常见的蓝牙耳机类型：\\n\\n* **入耳式蓝牙耳机：**这种耳机体积小巧，佩戴舒适，隔音效果好，适合运动和日常使用。\\n* **头戴式蓝牙耳机：**这种耳机佩戴舒适，音质较好，适合长时间佩戴和欣赏音乐。\\n* **骨传导蓝牙耳机：**这种耳机通过骨骼传递声音，无需入耳，适合运动和户外使用。\\n* **真无线蓝牙耳机：**这种耳机由两个独立的耳机组成，无需线缆连接，佩戴方便，适合运动和日常使用。\\n\\n选择蓝牙耳机时，需要考虑以下因素：\\n\\n* **音质：**音质是选择蓝牙耳机的重要因素，需要根据自己的需求选择合适的音质。\\n* **佩戴舒适度：**佩戴舒适度直接影响使用体验，需要选择适合自己的耳机类型。\\n* **续航时间：**续航时间决定了耳机的使用时间，需要根据自己的需求选择合适的续航时间。\\n* **功能：**一些蓝牙耳机具有降噪、防水等功能，需要根据自己的需求选择合适的功能。\\n\\n希望以上信息对您有所帮助。如果您有其他问题，请随时提问。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 273, 'prompt_tokens': 7, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'glm-4-air-250414', 'system_fingerprint': None, 'id': '202511102054044a9f215eb37147fd', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--5ee3a7dc-

In [13]:
# 中文
prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain = (prompt|llm)
product = "蓝牙耳机"
chain.invoke(product)

AIMessage(content='为制造蓝牙耳机的公司选择最佳名称，需要考虑几个因素：品牌形象、目标市场、产品特性以及名称的独特性和易记性。以下是一些不同风格和方向的名称建议，希望能给你带来灵感：\n\n**一、 突出科技感和创新性:**\n\n*   **NexusWave:** Nexus 意为连接点，Wave 指波形或蓝牙信号，体现连接和无线特性。\n*   **Synapse Audio:** Synapse 指神经元连接，暗示耳机的高效传输和清晰音质。\n*   **QuantumSound:** Quantum 指量子，体现尖端科技和卓越音质。\n*   **AetherAudio:** Aether 古希腊语中指以太，代表无形的媒介，契合无线传输的特性。\n*   **PulseTech:** Pulse 指脉搏或脉冲，体现动态和活力，Tech 则强调科技感。\n\n**二、 强调音质和听觉体验:**\n\n*   **ClarityWave:** 直接表达清晰音质和无线传输。\n*   **HarmonySound:** 强调和谐、悦耳的听觉体验。\n*   **EuphonyTech:** Euphony 指悦耳的声音，体现高品质音效。\n*   **MelodiaAudio:** Melodia 源自希腊语，意为旋律，强调音乐的优美。\n*   **SonicSculpt:** Sonic 指声音，Sculpt 意为雕刻，比喻对声音的精细处理。\n\n**三、 注重时尚和生活方式:**\n\n*   **AuraBeats:** Aura 指光环或氛围，Beats 强调节奏和动感，适合年轻时尚群体。\n*   **StyloFone:** Stylo 源自希腊语，意为风格，Fone 指电话或耳机，体现时尚与科技的结合。\n*   **VibeSound:** Vibe 指感觉或氛围，体现耳机的个性化和时尚感。\n*   **UrbanEars:** 强调都市生活方式和年轻潮流。\n*   **ChromaTune:** Chroma 指色彩，Tune 指调音，体现多彩外观和优质音效。\n\n**四、 简洁易记且具有独特性:**\n\n*   **Zeno:** 简洁有力，易于记忆，发音独特。\n*   **Kairos:** 希腊语，意为关键

In [10]:
aimsg = chain.invoke(product)

In [18]:
aimsg.messages[0].content

"描述制造content='蓝牙耳机是一种无线耳机，它使用蓝牙技术与音频源（如手机、电脑、平板等）进行连接，从而实现无线音频传输。蓝牙耳机具有便携、无线、方便等优点，广泛应用于日常生活、办公、运动、旅行等场景。\\n\\n蓝牙耳机的主要特点包括：\\n\\n1. 无线连接：通过蓝牙技术与音频源进行无线连接，无需担心线缆的束缚，使用更加自由。\\n\\n2. 便携性：蓝牙耳机通常体积小巧，便于携带，可以轻松放入口袋或包中。\\n\\n3. 蓝牙版本：蓝牙耳机支持不同的蓝牙版本，如蓝牙4.0、蓝牙5.0、蓝牙5.1等。蓝牙版本越高，传输速度越快，稳定性越好，功耗也相对较低。\\n\\n4. 电池续航：蓝牙耳机内置电池，续航时间因品牌、型号和功能而异。一般而言，蓝牙耳机的续航时间在几小时到几十小时之间。\\n\\n5. 噪音消除：部分蓝牙耳机具备主动降噪功能，可以有效降低环境噪音，提供更好的听觉体验。\\n\\n6. 触控操作：一些蓝牙耳机支持触控操作，用户可以通过触摸耳机来控制音乐播放、接听电话等。\\n\\n7. 多设备连接：部分蓝牙耳机支持同时连接多个设备，方便用户在不同设备间切换使用。\\n\\n8. 防水防尘：部分蓝牙耳机具备防水防尘功能，可以在一定程度上抵抗水滴和灰尘的侵入，适合运动或户外使用。\\n\\n9. 健康监测：部分蓝牙耳机具备心率监测、血氧监测等健康功能，可以帮助用户实时了解自己的身体状况。\\n\\n10. 音质：蓝牙耳机的音质因品牌、型号和价格而异。一般来说，价格较高的蓝牙耳机音质更好，但也有一些性价比较高的产品，可以满足基本需求。\\n\\n选择蓝牙耳机时，可以根据自己的需求和预算进行选择，关注蓝牙版本、续航时间、降噪功能、音质等方面。同时，也可以参考其他用户的评价和推荐，以获取更多信息。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 418, 'prompt_tokens': 7, 'total_tokens': 425, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'opena

## 2. Sequential Chains

### 2.1 SimpleSequentialChain

顺序链（Sequential Chains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入

In [ ]:
# from langchain_core.chains import SimpleSequentialChain

In [ ]:
# # OPENAI_API_KEY = "********"      #"填入你的专属的API key"
# llm = ChatOpenAI(temperature=0.9,openai_api_key=OPENAI_API_KEY)

子链 1

In [30]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=llm,
    
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='7456772f-1929-4b77-aa39-20ae0418183f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 113, 'total_tokens': 124, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'glm-4-air-250414', 'system_fingerprint': None, 'id': '2025111022300591c0299544594477', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d799fa61-0d96-4c28-9faa-cbd086826d58-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_-8170207124304652149', 'type': 'tool_call'}], usage_metadata={'input_tokens': 113, 'output_tokens': 11, 'total_tokens': 124, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='1cac78c6-4d15-4143-9a1b-ebfe58051b

子链 2

In [34]:

# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = (llm|second_prompt)

现在我们可以组合两个LLMChain，以便我们可以在一个步骤中创建公司名称和描述

In [35]:
overall_simple_chain = (chain_one|chain_two)

给一个输入，然后运行上chain_one面的链

In [36]:
product = "Queen Size Sheet Set"
overall_simple_chain.invoke(product)

ChatPromptValue(messages=[HumanMessage(content='Write a 20 words description for the following     company:content="It seems there might be a misunderstanding in your question. The text you provided describes a **Queen Size Sheet Set**, which is a bedding product, not a company. \\n\\nIf you\'re looking for a name for a company that makes bedding or home textiles, here are some suggestions:\\n\\n1. **ComfortCrest**\\n2. **SleepWell Designs**\\n3. **DreamWeave Home**\\n4. **Elegance Linens**\\n5. **CozyRest Textiles**\\n6. **SereneSleep Co.**\\n7. **Luxura Home**\\n8. **CalmCrest Bedding**\\n9. **PureSleep Essentials**\\n10. **RestfulNest Home**\\n\\nIf you have a specific focus or niche for the company, let me know, and I can tailor the suggestions accordingly!" additional_kwargs={\'refusal\': None} response_metadata={\'token_usage\': {\'completion_tokens\': 164, \'prompt_tokens\': 979, \'total_tokens\': 1143, \'completion_tokens_details\': None, \'prompt_tokens_details\': None}, \'mod

In [21]:
# 中文
first_prompt = ChatPromptTemplate.from_template(   
    "描述制造{product}的公司的最佳名称是什么?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(   
    "写一个20字的描述对于下面这个\
    公司：{company_name}的"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
product = "蓝牙耳机"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
蓝牙之音。
蓝牙之音，专业生产蓝牙音响的公司，提供高质量的无线音乐体验。

> Finished chain.


'蓝牙之音，专业生产蓝牙音响的公司，提供高质量的无线音乐体验。'

### 2.2 SequentialChain

当只有一个输入和一个输出时，简单的顺序链可以顺利完成。但是当有多个输入或多个输出时该如何实现呢？

我们可以使用普通的顺序链来实现这一点

In [ ]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="claude-sonnet-4-5-20250929",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

初始化语言模型

In [23]:
# OPENAI_API_KEY = "********"      #"填入你的专属的API key"
llm = ChatOpenAI(temperature=0.9,openai_api_key=OPENAI_API_KEY)

接下来我们将创建一系列的链，然后一个接一个使用他们

In [19]:
#子链1

# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: 输入：Review 输出： 英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


NameError: name 'LLMChain' is not defined

In [25]:
#子链2

# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: 输入：英文的Review   输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [26]:
#子链3

# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: 输入：Review  输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [27]:
#子链4

# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: 输入： 总结, 语言    输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [28]:
# 对四个子链进行组合

#输入：review    输出：英文review，总结，后续回复 
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

让我们选择一篇评论并通过整个链传递它，可以发现，原始review是法语，可以把英文review看做是一种翻译，接下来是根据英文review得到的总结，最后输出的是用法语原文进行的续写信息。

In [30]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre, and suspects it might be an old batch or counterfeit due to the foam not holding up.',
 'followup_message': "Réponse de suivi:\nJe vous remercie pour votre commentaire sur le goût du produit. Nous sommes désolés d'apprendre que cela n'a pas répondu à vos attentes. Nous prenons très au sérieux toute préoccupation concernant la qualité de nos produits et nous examinerons attentivement votre cas pour déterminer s'il s'agit d'un lot périmé ou contrefait. Nous apprécions votre feedback et nous nous efforcerons de faire en sorte que nos produits correspondent toujours à vos attente

## 3. Router Chain（路由链）

到目前为止，我们已经学习了LLM链和顺序链。但是，如果您想做一些更复杂的事情怎么办？

一个相当常见但基本的操作是根据输入将其路由到一条链，具体取决于该输入到底是什么。如果你有多个子链，每个子链都专门用于特定类型的输入，那么可以组成一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。

路由器由两个组件组成：

- 路由器链本身（负责选择要调用的下一个链）
- destination_chains：路由器链可以路由到的链

举一个具体的例子，让我们看一下我们在不同类型的链之间路由的地方，我们在这里有不同的prompt:  

### 定义提示模板

In [31]:
#第一个提示适合回答物理问题
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


#第二个提示适合回答数学问题
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""


#第三个适合回答历史问题
history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

首先需要定义这些提示模板，在我们拥有了这些提示模板后，可以为每个模板命名，然后提供描述。例如，第一个物理学的描述适合回答关于物理学的问题，这些信息将传递给路由链，然后由路由链决定何时使用此子链。

In [32]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

### 导入相关的包

### LLMRouterChain（此链使用 LLM 来确定如何路由事物）

在这里，我们需要一个**多提示链**。这是一种特定类型的链，用于在多个不同的提示模板之间进行路由。
但是，这只是你可以路由的一种类型。你也可以在任何类型的链之间进行路由。

这里我们要实现的几个类是LLM路由器链。这个类本身使用语言模型来在不同的子链之间进行路由。
这就是上面提供的描述和名称将被使用的地方。

#### 创建目标链  
目标链是由路由链调用的链，每个目标链都是一个语言模型链

#### 创建默认目标链
除了目标链之外，我们还需要一个默认目标链。这是一个当路由器无法决定使用哪个子链时调用的链。在上面的示例中，当输入问题与物理、数学、历史或计算机科学无关时，可能会调用它。

#### 创建LLM用于在不同链之间进行路由的模板
这包括要完成的任务的说明以及输出应该采用的特定格式。

In [31]:
from langchain.chat_models import init_chat_model

model = llm

def get_weather(location: str) -> str:
    """Get the weather at a location."""
    ...

model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Paris?")

for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"ID: {tool_call['id']}")

Tool: get_weather
Args: {'location': 'Paris'}
ID: call_-8170207089944911887


In [43]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse


basic_model = ChatOpenAI(model="glm-4-air-0111"
                         , api_key= os.getenv("ZAI_API_KEY")
                         ,openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
                        )
advanced_model = llm2

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

agent = create_agent(
    model=basic_model,  # Default model
    # tools=model_with_tools,
    middleware=[dynamic_model_selection]
)
agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me a joke"}]}
)

{'messages': [HumanMessage(content='Tell me a joke', additional_kwargs={}, response_metadata={}, id='1a93832e-6ada-4861-b569-b91e676df9e6'),
  AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 9, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'glm-4-air-250414', 'system_fingerprint': None, 'id': '202511102247277d5660553fc64a95', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--cff1d326-6d10-4045-8731-b94a472705b6-0', usage_metadata={'input_tokens': 9, 'output_tokens': 15, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}})]}